# Homework 2

Here we will implement a map-reduce algorithm in spark for counting the number of words from a given set of documents.
Let's start by properly initializing a SparkContext object.

In [1]:
from pyspark import SparkConf, SparkContext

config = SparkConf().setAppName('Homework 2').setMaster('local')

sc = SparkContext(conf=config)

### Loading the Dataset
Here we load the dataset and split it in a number of partitions. As a rule of thumb: the higher the number of partitions, the better the parallelism. One should nonetheless be aware that each partition have some overhead and thus it would be suboptimal to create a big number of partitions.

Since we're running on *4-core* machines we'll be partition the RDD in 8 parts.

In [2]:
docs = sc.textFile('text-sample.txt').persist()

### Trivial Algorithm
In this naive algorithm we do the following:
- Take each word from each document
- Create a new key-value pair for each word with a value 1
- Collect the pairs by key and sum their values

In [3]:
words = docs.flatMap(lambda document: document.split(" "))\
    .map(lambda word: (word,1))\
    .reduceByKey(lambda x,y: x+y)

print('The number of different words is: ', words.count())

The number of different words is:  144873


## Improved word count 1 - Gugio's, just used this as reference for the improved 2

In this first more clever version of word count we need to modify the way documents are processed: in particular, we want to store directly the pair *(w,c(w))* were _c(w)_ is the number of occurrences of the word *w* in the document Di

In [4]:
import time

def wordcount_0_function(document) :
    pairs_dict = {}
    for word in document.split(' '):
        if word not in pairs_dict.keys():
            pairs_dict[word] = 1
        else :
            pairs_dict[word] += 1
    return [(key, pairs_dict[key]) for key in pairs_dict.keys()]

 
wordcount_1 = docs.flatMap(wordcount_0_function) \
                .reduceByKey(lambda accum, n: accum + n)

t0 = time.time()
n_dif_words = wordcount_1.count()
t1 = time.time()
                
print('Improved WordCount 1 result:')
print('Number of different Words :', n_dif_words)
print('Elapsed Time :', t1-t0, 's')

Improved WordCount 1 result:
Number of different Words : 144873
Elapsed Time : 3.9227051734924316 s


## Improved word count 2 

In [5]:
N = docs.flatMap(lambda document: document.split(" "))\
    .map(lambda word: (word,1))\
    .count()
N

3503570

In [6]:
import numpy as np
import math

def wordcount_2(document) :
    pairs_dict = {}
    num_part =  math.floor(math.sqrt(N)) - 1
    for word in document.split(' '):
        if word not in pairs_dict.keys():
            pairs_dict[word] = 1
        else :
            pairs_dict[word] += 1
    return [(np.random.randint(0,num_part), (key, pairs_dict[key])) for key in pairs_dict.keys()]

def reduce_1(document):
    pairs_dict = {}
    for el in list(document[1]):
        if el[0] not in pairs_dict.keys():
            pairs_dict[el[0]] = el[1]
        else :
            pairs_dict[el[0]] += el[1]
    return [(key, pairs_dict[key]) for key in pairs_dict.keys()]

wordcount_2 = docs.flatMap(wordcount_2)\
    .groupByKey()\
    .flatMap(reduce_1)\
    .reduceByKey(lambda accum, n: accum + n)
    
t0 = time.time()
n_dif_words_2 = wordcount_2.count()
t1 = time.time()
                
print('Improved WordCount 2 result:')
print('Number of different Words :', n_dif_words_2)
print('Elapsed Time :', t1-t0, 's')

Improved WordCount 2 result:
Number of different Words : 144873
Elapsed Time : 9.016728639602661 s
